<h1>The apartment finder:</h1>

<h2>First, we ge the apartment information from our Kijiji scraper:</h2>

In [1]:
                                                                                                                                                                                                    #Get informations from Kijiji scraper:
%run kijiji_scraper.ipynb

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - There is no [linux64] chromedriver for browser 87.0.4280 in cache
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_linux64.zip


[WDM] - Driver has been saved in cache [/home/kurosuzume/.wdm/drivers/chromedriver/linux64/87.0.4280.88]


There are no more apartments with those filters right now. 
chrome closed


In [2]:
import html
from IPython.display import HTML


<h2>The apartment addresses are shown below.</h2>

In [3]:

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


<h2>Now, we import all libraries needed:</h2>

In [4]:
import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json #library to handle JSON files

!pip install pgeocode
import pgeocode

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't competed 
from geopy.geocoders import Nominatim

#Matplotlib and associated ploting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium-0.5.0 --yes
import folium

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

from geopy import distance  
print("Libraries imported successfully")



Solving environment: done

## Package Plan ##

  environment location: /home/kurosuzume/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    certifi-2020.12.5          |   py37h89c1867_1         143 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         344 KB

The following packages will be UPDATED:

  ca-certificates                      2020.11.8-ha878542_0 --> 2020.12.5-ha878542_0
  certifi                          2020.11.8-py37h89c1867_0 --> 2020.12.5-py37h89c1867_1
  geopy                                  2.0.0-pyh9f0ad1d_0 --> 2.1.0-pyhd3deb0d_0



ca-certificates-2020 | 137 KB    | #################

<h2>The reference point is the NBCC campus in New Brunswick.</h2>
<h3>We set the location coordinates and the radius we will use to find the apartments and amenities:</h3>

In [5]:
address = '"26 Duffie Dr, Fredericton, NB , Canada"'

geolocator = Nominatim(user_agent="NB_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of the NBCC Fredericton campus are {}, {}.".format(latitude, longitude))
radius = 10000


The geographical coordinates of the NBCC Fredericton campus are 45.9428786, -66.6456084.


<h3>Ensuring all te data is in the right format:</h3>

In [6]:
#Before finding the geo-locations of the addresses given, we need to verify if they are in the right format:
main_address = []
unaccepted_address_index = []

for address in text_address:
    house = address.split(',')
    house = house[0]+",Fredericton, CA"
    main_address.append(house)

for i in range(len(main_address)):
    if main_address is None:    
        main_address.remove(main_address[i])
        ap_prices.remove(ap_prices[i])
        title_pic_change.remove(title_pic_change[i])
        num_bedrooms.remove(num_bedrooms[i])
        num_bathrooms.remove(num_bathrooms[i])

print("Data format verified!")

Data format verified!


In [7]:
geo_location = main_address

<h2>Finding the geographic location of each apartment:</h2>

In [8]:
#Now, to find the geo location of each of the addresses:

index_to_remove = []
geo_location = []
for address in range(len(main_address)):
    appended_location = geolocator.geocode(main_address[address])
    geo_location.append(appended_location)
    if appended_location is None:
        index_to_remove.append(address)
index_to_remove = index_to_remove.reverse()

if index_to_remove is None:
        pass
else: 
    for i in index_to_remove:

        geo_location.remove(geo_location[address])
        ap_prices.remove(ap_prices[address])
        num_bedrooms.remove(num_bedrooms[address])
        num_bathrooms.remove(num_bathrooms[address])          

print("geo location acquired.")

geo location acquired.


<h3>Showing the Location information for all the apartments</h3>

In [9]:
geo_location

[None,
 Location(825, McLeod Avenue, Queen's Park, Downtown, Fredericton, York County, New Brunswick / Nouveau-Brunswick, E3B 5H5, Canada, (45.9517079, -66.64022605005037, 0.0)),
 None,
 None,
 Location(Fredericton, Saint Marys Parish, New Brunswick / Nouveau-Brunswick, Canada, (45.947959350000005, -66.65336235897576, 0.0)),
 None,
 Location(York Street, Garrison District, Downtown, Fredericton, York County, New Brunswick / Nouveau-Brunswick, E3B 1B5, Canada, (45.9632279, -66.642953, 0.0)),
 Location(300, Inglewood Drive, Sunshine Gardens, Fredericton, York County, New Brunswick / Nouveau-Brunswick, E3B 2L4, Canada, (45.9604331, -66.6664044, 0.0)),
 Location(Fredericton, Saint Marys Parish, New Brunswick / Nouveau-Brunswick, Canada, (45.947959350000005, -66.65336235897576, 0.0)),
 None,
 Location(242, Regent Street, Capital, Downtown, Fredericton, York County, New Brunswick / Nouveau-Brunswick, E3B 3W4, Canada, (45.95741875, -66.64155004210608, 0.0)),
 None,
 None,
 None,
 Location(376

In [10]:
location_info = []
discarded_index = []
for location in range(len(geo_location)):
    if not geo_location[location] == None: 
        location_info.append([geo_location[location].address, geo_location[location].latitude, geo_location[location].longitude ])
    else: 
        discarded_index.append(location)
    

<h3>Using the Introduction_to_the_Problem notebook to retrieve the map.</h3>

In [11]:
#Get the map code from the introduction notebook:
%run Introduction_to_the_Problem.ipynb

The college address is: Duffie Drive, College Hill, Skyline Acres, Fredericton, York County, New Brunswick / Nouveau-Brunswick, E3B 9W4, Canada


In [12]:
address = '"26 Duffie Dr, Fredericton, NB , Canada"'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of the NBCC Fredericton campus are {}, {}.".format(latitude, longitude))
radius = 10000



The geographical coordinates of the NBCC Fredericton campus are 45.9428786, -66.6456084.


<h3>Foursquare credentials:</h3>
<p>Please, fill with your foursquare information:</p>

In [13]:
CLIENT_ID = 'GRUEM4VGSOO2HFMKCVFHP155RMR0VF4B0G1XZF3XSD5IIUQ4' # your Foursquare ID
CLIENT_SECRET = 'HLZAHXL4C431II4Q1AX5LX2NMW1EFAEUTXBSHCBKQLPL5POG' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID + 'PLEASE VERIFY!')
print('CLIENT_SECRET:' + CLIENT_SECRET + 'PLEASE VERIFY!')

Your credentails:
CLIENT_ID: GRUEM4VGSOO2HFMKCVFHP155RMR0VF4B0G1XZF3XSD5IIUQ4PLEASE VERIFY!
CLIENT_SECRET:HLZAHXL4C431II4Q1AX5LX2NMW1EFAEUTXBSHCBKQLPL5POGPLEASE VERIFY!


<p>Since it wasn't possible to use an automatic iterative method to find the amenities, we will do it manually. for that we will use the Foursquare API and acquire nearby venues data:</p>

In [14]:
#Now, to get the nearby venues according to the important amenities:
amenities_list = ["Grocery Store","Supermarket", "Gym", "Drugstore", "Hospital", "Candy Store"]
search_query0 = "Grocery Store"
search_query1 = "Supermarket"
search_query2 = "Gym"
search_query3 = "Drugstore"
search_query4 = "Hospital"
search_query5 = "Candy Store"


In [15]:
url0 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query0, radius, LIMIT)
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query1, radius, LIMIT)
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query2, radius, LIMIT)
url3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query3, radius, LIMIT)
url4 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query4, radius, LIMIT)
url5 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query5, radius, LIMIT)


In [16]:
results0 = requests.get(url0).json()
results1 = requests.get(url1).json()
results2 = requests.get(url2).json()
results3 = requests.get(url3).json()
results4 = requests.get(url4).json()
results5 = requests.get(url5).json()



In [17]:
# assign relevant part of JSON to venues
venues0 = results0['response']['venues']
venues1 = results1['response']['venues']
venues2 = results2['response']['venues']
venues3 = results3['response']['venues']
venues4 = results4['response']['venues']
venues5 = results5['response']['venues']

# transform venues into dataframes
dataframe0 = pd.json_normalize(venues0)
dataframe1 = pd.json_normalize(venues1)
dataframe2 = pd.json_normalize(venues2)
dataframe3 = pd.json_normalize(venues3)
dataframe4 = pd.json_normalize(venues4)
dataframe5 = pd.json_normalize(venues5)


In [18]:
print("Venues Data retrieved.")

Venues Data retrieved.


<h2>Results from foursquare will converge into one dataframe:</h2>

In [19]:
# now converge all results into one dataframe:

dataframe0 = pd.concat([dataframe0,dataframe1], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe2], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe3], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe4], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe5], axis=0, ignore_index=True)



dataframe0.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,4c545e5db426ef3b11cc7e8a,Dick's Grocery Store,"[{'id': '4bf58dd8d48988d123951735', 'name': 'S...",v-1610228994,False,480 Westmorland St,45.941957,-66.663877,"[{'label': 'display', 'lat': 45.94195715102332...",1417,E3B 3M9,CA,Fredericton,NB,Canada,"[480 Westmorland St, Fredericton NB E3B 3M9, C...",NaN,NaN
1,4c3c8e25282203bb55a3fcda,American Eagle & Aerie Store,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",v-1610228994,False,1381 Regent Street,45.935129,-66.664601,"[{'label': 'display', 'lat': 45.93512887044409...",1704,E3C 1A2,CA,Fredericton,NB,Canada,"[1381 Regent Street, Fredericton NB E3C 1A2, C...",NaN,NaN
2,4badfab8f964a520a3773be3,Dollar Store,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",v-1610228994,False,NaN,45.942453,-66.650432,"[{'label': 'display', 'lat': 45.94245279861792...",376,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN
3,53f7c437498eacb60d222490,Great Canadian Dollar Store,"[{'id': '52dea92d3cf9994f4e043dbb', 'name': 'D...",v-1610228994,False,Uptown Center,45.939575,-66.663973,"[{'label': 'display', 'lat': 45.93957493230641...",1468,NaN,CA,Fredericton,NB,Canada,"[Uptown Center (Prospect St), Fredericton NB, ...",Prospect St,NaN
4,505ba09be4b027df9b199de0,Jump +,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E...",v-1610228994,False,1381 Regent St,45.935711,-66.664615,"[{'label': 'display', 'lat': 45.93571105673426...",1673,E3C 1A2,CA,Fredericton,NB,Canada,"[1381 Regent St, Fredericton NB E3C 1A2, Canada]",NaN,NaN


In [20]:
dataframe0.shape

(96, 18)

<h2>Filtering the pandas dataframe:</h2>

In [21]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe0.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe0.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Dick's Grocery Store,Smoke Shop,480 Westmorland St,45.941957,-66.663877,"[{'label': 'display', 'lat': 45.94195715102332...",1417,E3B 3M9,CA,Fredericton,NB,Canada,"[480 Westmorland St, Fredericton NB E3B 3M9, C...",NaN,4c545e5db426ef3b11cc7e8a
1,American Eagle & Aerie Store,Clothing Store,1381 Regent Street,45.935129,-66.664601,"[{'label': 'display', 'lat': 45.93512887044409...",1704,E3C 1A2,CA,Fredericton,NB,Canada,"[1381 Regent Street, Fredericton NB E3C 1A2, C...",NaN,4c3c8e25282203bb55a3fcda
2,Dollar Store,Miscellaneous Shop,NaN,45.942453,-66.650432,"[{'label': 'display', 'lat': 45.94245279861792...",376,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4badfab8f964a520a3773be3
3,Great Canadian Dollar Store,Discount Store,Uptown Center,45.939575,-66.663973,"[{'label': 'display', 'lat': 45.93957493230641...",1468,NaN,CA,Fredericton,NB,Canada,"[Uptown Center (Prospect St), Fredericton NB, ...",Prospect St,53f7c437498eacb60d222490
4,Jump +,Electronics Store,1381 Regent St,45.935711,-66.664615,"[{'label': 'display', 'lat': 45.93571105673426...",1673,E3C 1A2,CA,Fredericton,NB,Canada,"[1381 Regent St, Fredericton NB E3C 1A2, Canada]",NaN,505ba09be4b027df9b199de0


In [22]:
amenities= dataframe_filtered["name"].to_list()
amenities_categories = dataframe_filtered["categories"].to_list()
amenities_list_lat = dataframe_filtered["lat"].to_list()
amenities_list_lng = dataframe_filtered["lng"].to_list()


<h3>And finally, we add the amenities to our Folium map.</h3>
<p>The amenities will be displayed as a blue dot with brown outline. </p>

In [23]:

for i in range(len(geo_location)):
    if not geo_location[i] == None: 
        tooltip = "Apartment"
        folium.Marker([geo_location[i].latitude,geo_location[i].longitude], popup=AD_title[i], tooltip = tooltip ).add_to(map_Fred)
    else:
        pass

In [24]:

# add the Italian amenities as blue circle markers with red outline
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='brown',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6,
        tooltip = label
    ).add_to(map_Fred)
map_Fred

<h3>Now, using geopy, we can verify amenities within a 800m radius from an apartment.</h3>

In [25]:
from geopy import point
#Function to find amenities in a 500m radius from a center point:
def find_in_radius(center_name, center_lat,center_lng, amenities_list_lat, amenities_list_lng, amenities_list, amenities_categories):
    """(float, float, list) -> list
        Retruns a list of amenities within a 800 radius from a center point.
    """
    center_lat = str(center_lat)
    center_lng = str(center_lng)
    center_point = point.Point(str(center_lat)+ "," + str(center_lng))
    
    within_radius = []
    amenities_cat = []
    for amenity in range(len(amenities_list)):
        amenities_list_point = point.Point(str(amenities_list_lat[amenity])+ "," + str(amenities_list_lng[amenity]))
        amen_dis = distance.distance(center_point, amenities_list_point).km
        if amen_dis <= 0.80: #1000 meters
            within_radius.append(amenities_list[amenity])
            amenities_cat.append(amenities_categories[amenity])
    return center_name,within_radius,amenities_cat

        

In [26]:
for type in range(len(amenities_categories)):
    if amenities_categories[type] == None:
        amenities_categories[type] = "None"

In [27]:
full_list = []
for i in range(len(location_info)):
    full_list.append(find_in_radius(location_info[i][0],location_info[i][1], location_info[i][2], amenities_list_lat, amenities_list_lng, amenities,amenities_categories))

In [28]:
reverse_discarded_index = discarded_index.reverse()

In [29]:
for i in discarded_index:
    ap_prices.remove(ap_prices[i])
    num_bedrooms.remove(num_bedrooms[i])
    num_bathrooms.remove(num_bathrooms[i])

In [30]:
ap_prices_float = []
for i in range(len(ap_prices)):
    split = ap_prices[i].replace('$', '')
    
    ap_prices_float.append(split)
    

In [31]:
ap_prices_final = []
for i in range(len(ap_prices_float)):
    make_float = float(ap_prices_float[i].replace(',',''))
    ap_prices_final.append(make_float)
ap_prices = ap_prices_final

In [32]:
apt_address = []
amenity_name = []
amenity_type= []
for i in range(len(full_list)):
    apt_address.append(full_list[i][0])
    amenity_name.append(full_list[i][1])
    amenity_type.append(full_list[i][2])

In [33]:
separator = ","
prepared_amenity = []
for i in range(len(amenity_type)):
    sep = separator.join(amenity_type[i])
    prepared_amenity.append(sep)


In [34]:
for i in range(len(prepared_amenity)):
    prepared_amenity[i] = str(prepared_amenity[i]).split(',')
    for j in range(len(prepared_amenity[i])):
        prepared_amenity[i][j].split(",")

    

In [35]:
apartment_df= pd.DataFrame(list(zip(apt_address, amenity_name, prepared_amenity, ap_prices, num_bedrooms, num_bathrooms)))

In [36]:
len(prepared_amenity)
result = list(set(x for l in prepared_amenity for x in l))


In [37]:
apartment_df = pd.concat([apartment_df,pd.get_dummies(apartment_df[2].apply(pd.Series).stack()).sum(level=0)], axis = 1)

In [38]:
apartment_df = apartment_df.replace({5:{'Bathrooms: 1': 1, 'Bathrooms: 2': 2, "Bathrooms: 1.5": 2}})

apartment_df = apartment_df.replace({4:{'Bedrooms: Bachelor/Studio': 0, 'Bedrooms: 1': 1,'Bedrooms: 2': 2, 'Bedrooms: 1 + Den':2}})


In [39]:
apartment_df = apartment_df.rename(columns={0: "Apartment Address", 1: "Amenities",2:"Amenity type", 3:"Rent price", 4:"Bedrooms", 5:"Bathrooms"})


In [40]:
from datetime import date
date.fromisoformat('2019-12-04')


datetime.date(2019, 12, 4)

#print(discarded_index)
#print(len(ap_prices))<h3>with all that information, we're able to create the apartment DataFrame:</h3>

In [41]:
for i in range(len(apartment_df)):
    apartment_df['date'] = date.today()
    

In [42]:
apartment_df.head(1)

,Apartment Address,Amenities,Amenity type,Rent price,Bedrooms,Bathrooms,,Candy Store,Clothing Store,College Gym,College Rec Center,Department Store,Discount Store,Furniture / Home Store,General Travel,Gym / Fitness Center,Hospital,Miscellaneous Shop,None,Nudist Beach,Paper / Office Supplies Store,Smoke Shop,Supermarket,Tourist Information Center,Veterinarian,date
0,"825, McLeod Avenue, Queen's Park, Downtown, Fr...","[The UPS Store, UPS Store, University Cafe & M...","[Miscellaneous Shop, Paper / Office Supplies S...",270.0,0,1,0,2,0,2,0,0,0,0,0,1,0,2,0,1,2,0,0,0,0,2021-01-09


In [43]:
apartment_df.shape

(22, 26)

In [44]:
apartment_df[''] = apartment_df.rename(columns={'':'empty'})

<h3>Then, we open the csv file with an older instance of the DF, and append the unique new values between them</h3>

In [45]:
apartment_df_initial = pd.read_csv("apartment_dataframe.csv", header = 0, index_col = False)


In [46]:
apartment_df_initial = apartment_df_initial.rename(columns={'Unnamed: 6':'empty'})

In [47]:
apartment_df_initial

,Apartment Address,Amenities,Amenity type,Rent price,Bedrooms,Bathrooms,empty,Clothing Store,College Gym,College Rec Center,Convenience Store,Department Store,Discount Store,Furniture / Home Store,General Travel,Gym,Gym / Fitness Center,Hardware Store,Hospital,Miscellaneous Shop,Paper / Office Supplies Store,Smoke Shop,Thrift / Vintage Store,Tourist Information Center,Veterinarian,date,Unnamed: 26,Grocery Store,Nudist Beach,Unnamed: 29,Electronics Store,IT Services,Mobile Phone Shop,Supermarket,Unnamed: 34
0,"Priestman Street, Rabbit Town, Uptown, Frederi...","[""Dick's Grocery Store"", 'Great Canadian Dolla...","['Smoke Shop', 'Discount Store', 'Department S...",450.0,0,1,0.0,0,0.0,0.0,0,0.0,2,2,0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"George Street, Carleton, Downtown, Fredericton...","['Urban Almanac', 'UPS Store', 'The UPS Store'...","['Furniture / Home Store', 'Paper / Office Sup...",500.0,0,1,0.0,0,2.0,0.0,0,2.0,0,0,2,2.0,1.0,0.0,2.0,0.0,1.0,2.0,0.0,2,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"George Street, Carleton, Downtown, Fredericton...","['Urban Almanac', 'UPS Store', 'The UPS Store'...","['Furniture / Home Store', 'Paper / Office Sup...",550.0,0,1,0.0,0,2.0,0.0,0,2.0,0,0,2,2.0,1.0,0.0,2.0,0.0,1.0,2.0,0.0,2,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"York Street, Rabbit Town, Downtown, Fredericto...","['UPS Store', 'The UPS Store', 'Kwik-Way Conve...","['Paper / Office Supplies Store', 'Miscellaneo...",600.0,0,1,0.0,0,0.0,0.0,0,2.0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Saint John Street, Queen's Park, Downtown, Fre...","['UPS Store', 'University Cafe & Market Conven...","['Paper / Office Supplies Store', 'Candy Store...",600.0,0,1,0.0,2,2.0,0.0,0,0.0,0,0,0,2.0,1.0,0.0,2.0,0.0,1.0,2.0,0.0,0,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"31, Corbett Avenue, Station Pointe, Barker's P...",[],[''],700.0,0,1,1.0,0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"36-38, Waterloo Row, Town Platt, Downtown, Fre...","['University Cafe & Market Convenience Store',...","['Candy Store', 'General Travel', 'Clothing St...",748.0,0,1,0.0,2,2.0,0.0,0,0.0,0,0,0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"706, Valleyview Street, College Hill, Skyline ...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","['Miscellaneous Shop', 'Gym / Fitness Center',...",750.0,1,1,0.0,0,0.0,2.0,1,0.0,0,0,0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Valleyview Street, College Hill, Skyline Acres...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","['Miscellaneous Shop', 'Gym / Fitness Center',...",750.0,1,1,0.0,0,0.0,2.0,1,0.0,0,0,0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Woodstock Road, Garden Creek, Fredericton, Yor...",[],[''],775.0,1,1,1.0,0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
apartment_df_initial.shape

(100, 35)

In [49]:
apartment_df_concat = pd.concat([apartment_df_initial,apartment_df]).reset_index(drop=True)

In [50]:
apartment_df_concat.shape

(122, 38)

In [51]:
apartment_df_concat

,Apartment Address,Amenities,Amenity type,Rent price,Bedrooms,Bathrooms,empty,Clothing Store,College Gym,College Rec Center,Convenience Store,Department Store,Discount Store,Furniture / Home Store,General Travel,Gym,Gym / Fitness Center,Hardware Store,Hospital,Miscellaneous Shop,Paper / Office Supplies Store,Smoke Shop,Thrift / Vintage Store,Tourist Information Center,Veterinarian,date,Unnamed: 26,Grocery Store,Nudist Beach,Unnamed: 29,Electronics Store,IT Services,Mobile Phone Shop,Supermarket,Unnamed: 34,,Candy Store,None
0,"Priestman Street, Rabbit Town, Uptown, Frederi...","[""Dick's Grocery Store"", 'Great Canadian Dolla...","['Smoke Shop', 'Discount Store', 'Department S...",450.0,0,1,0.0,0,0.0,0.0,0.0,0.0,2,2,0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"George Street, Carleton, Downtown, Fredericton...","['Urban Almanac', 'UPS Store', 'The UPS Store'...","['Furniture / Home Store', 'Paper / Office Sup...",500.0,0,1,0.0,0,2.0,0.0,0.0,2.0,0,0,2,2.0,1.0,0.0,2.0,0.0,1.0,2.0,0.0,2,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"George Street, Carleton, Downtown, Fredericton...","['Urban Almanac', 'UPS Store', 'The UPS Store'...","['Furniture / Home Store', 'Paper / Office Sup...",550.0,0,1,0.0,0,2.0,0.0,0.0,2.0,0,0,2,2.0,1.0,0.0,2.0,0.0,1.0,2.0,0.0,2,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"York Street, Rabbit Town, Downtown, Fredericto...","['UPS Store', 'The UPS Store', 'Kwik-Way Conve...","['Paper / Office Supplies Store', 'Miscellaneo...",600.0,0,1,0.0,0,0.0,0.0,0.0,2.0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Saint John Street, Queen's Park, Downtown, Fre...","['UPS Store', 'University Cafe & Market Conven...","['Paper / Office Supplies Store', 'Candy Store...",600.0,0,1,0.0,2,2.0,0.0,0.0,0.0,0,0,0,2.0,1.0,0.0,2.0,0.0,1.0,2.0,0.0,0,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"31, Corbett Avenue, Station Pointe, Barker's P...",[],[''],700.0,0,1,1.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"36-38, Waterloo Row, Town Platt, Downtown, Fre...","['University Cafe & Market Convenience Store',...","['Candy Store', 'General Travel', 'Clothing St...",748.0,0,1,0.0,2,2.0,0.0,0.0,0.0,0,0,0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"706, Valleyview Street, College Hill, Skyline ...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","['Miscellaneous Shop', 'Gym / Fitness Center',...",750.0,1,1,0.0,0,0.0,2.0,1.0,0.0,0,0,0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Valleyview Street, College Hill, Skyline Acres...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","['Miscellaneous Shop', 'Gym / Fitness Center',...",750.0,1,1,0.0,0,0.0,2.0,1.0,0.0,0,0,0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Woodstock Road, Garden Creek, Fredericton, Yor...",[],[''],775.0,1,1,1.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
#apartment_df_concat = apartment_df_concat.drop(columns=['', 'Nudist Beach'])

In [53]:
apartment_df = apartment_df_concat.drop_duplicates(keep='first', subset=['Apartment Address', 'Rent price'])

In [54]:
#apartment_df = apartment_df.drop(['Unnamed: 0'], axis=1)

In [55]:
apartment_df

,Apartment Address,Amenities,Amenity type,Rent price,Bedrooms,Bathrooms,empty,Clothing Store,College Gym,College Rec Center,Convenience Store,Department Store,Discount Store,Furniture / Home Store,General Travel,Gym,Gym / Fitness Center,Hardware Store,Hospital,Miscellaneous Shop,Paper / Office Supplies Store,Smoke Shop,Thrift / Vintage Store,Tourist Information Center,Veterinarian,date,Unnamed: 26,Grocery Store,Nudist Beach,Unnamed: 29,Electronics Store,IT Services,Mobile Phone Shop,Supermarket,Unnamed: 34,,Candy Store,None
0,"Priestman Street, Rabbit Town, Uptown, Frederi...","[""Dick's Grocery Store"", 'Great Canadian Dolla...","['Smoke Shop', 'Discount Store', 'Department S...",450.0,0,1,0.0,0,0.0,0.0,0.0,0.0,2,2,0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"George Street, Carleton, Downtown, Fredericton...","['Urban Almanac', 'UPS Store', 'The UPS Store'...","['Furniture / Home Store', 'Paper / Office Sup...",500.0,0,1,0.0,0,2.0,0.0,0.0,2.0,0,0,2,2.0,1.0,0.0,2.0,0.0,1.0,2.0,0.0,2,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"George Street, Carleton, Downtown, Fredericton...","['Urban Almanac', 'UPS Store', 'The UPS Store'...","['Furniture / Home Store', 'Paper / Office Sup...",550.0,0,1,0.0,0,2.0,0.0,0.0,2.0,0,0,2,2.0,1.0,0.0,2.0,0.0,1.0,2.0,0.0,2,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"York Street, Rabbit Town, Downtown, Fredericto...","['UPS Store', 'The UPS Store', 'Kwik-Way Conve...","['Paper / Office Supplies Store', 'Miscellaneo...",600.0,0,1,0.0,0,0.0,0.0,0.0,2.0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Saint John Street, Queen's Park, Downtown, Fre...","['UPS Store', 'University Cafe & Market Conven...","['Paper / Office Supplies Store', 'Candy Store...",600.0,0,1,0.0,2,2.0,0.0,0.0,0.0,0,0,0,2.0,1.0,0.0,2.0,0.0,1.0,2.0,0.0,0,1,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"31, Corbett Avenue, Station Pointe, Barker's P...",[],[''],700.0,0,1,1.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"36-38, Waterloo Row, Town Platt, Downtown, Fre...","['University Cafe & Market Convenience Store',...","['Candy Store', 'General Travel', 'Clothing St...",748.0,0,1,0.0,2,2.0,0.0,0.0,0.0,0,0,0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"706, Valleyview Street, College Hill, Skyline ...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","['Miscellaneous Shop', 'Gym / Fitness Center',...",750.0,1,1,0.0,0,0.0,2.0,1.0,0.0,0,0,0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Valleyview Street, College Hill, Skyline Acres...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","['Miscellaneous Shop', 'Gym / Fitness Center',...",750.0,1,1,0.0,0,0.0,2.0,1.0,0.0,0,0,0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Woodstock Road, Garden Creek, Fredericton, Yor...",[],[''],775.0,1,1,1.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
apartment_df.to_csv("apartment_dataframe.csv",mode='w', header=True, index=False)

print("The apartment_dataframe.csv has been updated!")

The apartment_dataframe.csv has been updated!


<p>Now, we will use this information to analyse the results and generate a conclusion.</p>